In [ ]:
import pixiedust
import numpy
import statistics as st

In [ ]:
%cd ..
%run visualisation.ipynb
%run common.ipynb

In [ ]:
vertices_path = "../Data/Basketball/Players.csv"
edges_path = "../Data/Basketball/Edges.csv"

nba11_20 = read_all_star_sample(vertices_path, edges_path, 10, True, True, True)
nba15_20 = read_all_star_sample(vertices_path, edges_path, 6, True, True, True)
nba19_20 = read_all_star_sample(vertices_path, edges_path, 2, True, True, True)

In [ ]:
def build_lpa(graph, output):
    result = graph.community_label_propagation()
    resultArray = result.membership
    i = 0
    for v in graph.vs:
        v["communities"] = [resultArray[i]]
        i += 1

    plot_full(graph, output, 1)

In [ ]:
build_lpa(nba11_20, "all_stars_1120.svg")
build_lpa(nba15_20, "all_stars_1520.svg")
build_lpa(nba19_20, "all_stars_1920.svg")

In [ ]:
def get_date_modification_index(vertex, max_indices):
    if len(max_indices) <= 1:
        return max_indices[0]
    
    dates = []
    for i in max_indices:
        dates.append([])
        
    for e in vertex.all_edges():
        com = 0
        if vertex == e.target_vertex and vertex == e.source_vertex:
            print("Warn - out vertex equals current vertex")
        elif vertex != e.target_vertex:
            com = e.target_vertex["communities"][0]
        elif vertex != e.source_vertex:
            com = e.source_vertex["communities"][0]
            
        if com in max_indices:
            dates[max_indices.index(com)].append(e["weight"])
            
    for idx, l in enumerate(dates):
        if len(l) > 1:
            dates[idx] = st.stdev(l)
        else:
            dates[idx] = 1
        
    min_index = dates.index(min(dates))
    return max_indices[min_index]    

In [ ]:
def get_mutual_modification_index(vertex, max_indices):
    if len(max_indices) <= 1:
        return max_indices[0]
    
    mutual = []
        
    for e in vertex.all_edges():
        com = 0
        if vertex == e.target_vertex and vertex == e.source_vertex:
            print("Warn - out vertex equals current vertex")
        elif vertex != e.target_vertex:
            com = e.target_vertex["communities"][0]
        elif vertex != e.source_vertex:
            com = e.source_vertex["communities"][0]
            
        if com in max_indices:
            mutual[max_indices.index(com)] += e["friends"]
        
    max_index = mutual.index(max(mutual))
    return max_indices[max_index]

In [ ]:
def my_basic_lpa(graph, max_iterations, date_modification, mutual_modification, output):
    print("Basic lpa, begin...")
    counters = []
    vertices_count = len(graph.vs)
    
    i = 0
    for v in graph.vs:
        v["communities"] = [i]
        i += 1
        counters.append(0)
    
    convergence = False
    iteration = 0
    while convergence == False and iteration < max_iterations:
        iteration += 1
        convergence = True
        
        for idx in range(vertices_count):
            counters[idx] = 0
            
        for v in graph.vs:
            com = v["communities"][0]
            counters[com] += 1
            
        unique = sum(1 if counter > 0 else 0 for counter in counters)
        print("Iteration " + str(iteration) + ",communities " + str(unique))

        for v in graph.vs:
            current_community = v["communities"][0]   
            
            #clear counters
            for idx in range(vertices_count):
                counters[idx] = 0
            
            for e in v.all_edges():     
                if v == e.target_vertex and v == e.source_vertex:
                    print("Warn - out vertex equals current vertex")
                elif v != e.target_vertex:
                    com = e.target_vertex["communities"][0]
                    counters[com] += 1
                elif v != e.source_vertex:
                    com = e.source_vertex["communities"][0]
                    counters[com] += 1
                    
            if sum(counters) != v.degree():
                print("Warn: sum = " + str(sum(counters)) + " degree = " + str(v.degree()))
                
            max_value = max(counters)
            max_indices = []
            
            for k in range(vertices_count):
                if (counters[k] == max_value):
                    max_indices.append(k)
            
            winner_idx = max_indices[random.randrange(len(max_indices))]
            if date_modification:
                winner_idx = get_date_modification_index(v, max_indices)
            if mutual_modification:
                winner_idx = get_mutual_modification_index(v, max_indices)
        
            if current_community in max_indices:
                winner_idx = current_community
                
            if winner_idx != current_community:
                convergence = False
                
            v["communities"] = [winner_idx]
            
    plot_full(graph, output, 1.5)

In [ ]:
my_basic_lpa(nba11_20, 10, False, True, "all_stars_my_1120.svg")
my_basic_lpa(nba15_20, 10, False, True, "all_stars_my_1520.svg")
my_basic_lpa(nba19_20, 10, False, True, "all_stars_my_1920.svg")

In [ ]:
def my_basic_lpa_overlapping(graph, max_communities, max_iterations, output):
    print("Basic lpa with overlapping communities, begin...")
    counters = []
    vertices_count = len(graph.vs)
    
    i = 0
    for v in graph.vs:
        v["communities"] = [(i, 1)]
        i += 1
        counters.append(0)
    
    convergence = False
    iteration = 0
    while convergence == False and iteration < max_iterations:
        iteration += 1
        convergence = True
        
        for idx in range(vertices_count):
            counters[idx] = 0
            
        for v in graph.vs:
            for com, weight in v["communities"]:
                counters[com] += weight
            
        unique = sum(1 if counter > 0 else 0 for counter in counters)
        print("Iteration " + str(iteration) + ",communities " + str(unique))

        for v in graph.vs:
            current_communities = v["communities"]   
            
            #clear counters
            for idx in range(vertices_count):
                counters[idx] = 0
            
            for e in v.all_edges():     
                if v == e.target_vertex and v == e.source_vertex:
                    print("Warn - out vertex equals current vertex")
                elif v != e.target_vertex:
                    for com, weight in e.target_vertex["communities"]:
                        counters[com] += weight
                elif v != e.source_vertex:
                    for com, weight in e.target_vertex["communities"]:
                        counters[com] += weight
                
            max_value = max(counters)
            sorted_counters = numpy.argsort(counters)[::-1]
            new_communities = []
            
            idx = 0
            
            if max_value == 0:
                continue
                
            normalize_factor = sum(counters)
            while counters[sorted_counters[idx]] / normalize_factor >= 1 / max_communities:
                new_communities.append((sorted_counters[idx], counters[sorted_counters[idx]] / max_value))
                idx += 1
            
            if idx == 0:
                new_communities = current_communities
               
            if len(list(set(new_communities) - set(current_communities))) > 0:
                convergence = False
                
            weight_sum = sum(w for com, w in new_communities)
            for i in range(len(new_communities)):
                new_communities[i] = (new_communities[i][0], new_communities[i][1] / weight_sum)
                
            v["communities"] = new_communities
            
    
    for v in graph.vs:
        print(str(v["communities"]))
        normalized = []
        for com, _ in v["communities"]:
            normalized.append(com)
        v["communities"] = normalized
        
    plot_full(graph, output, 1.5)

In [ ]:
my_basic_lpa_overlapping(nba19_20, 4, 10, "all_stars_myo4_1920.svg")
my_basic_lpa_overlapping(nba11_20, 4, 10, "all_stars_myo4_1120.svg")
my_basic_lpa_overlapping(nba15_20, 4, 10, "all_stars_myo4_1520.svg")